### **Least square classifier from scratch**

We will implement least square classification from scratch in this notebook.

*A few points to recall from the theory:*

* Least square classification is used for estimating parameters of discriminant function from the given training data.

* Least square classification adapts linear regression model for classification.

 * It uses **least square error** as a **loss funciton**

 * It uses **normal equation** method and **gradient descent** for estimating model parameters or weight vector.

* Since it is a classification algorithm, we would use classification related evaluation metrics such as precision, recall, F-1 score ,AUC ROC/PR, and accuracy.

*Additionally note that:*

* We make use of polynomial feature transformation to obtain new features and then use that representation to learn non-linear decision boundaries between classes.

\begin{equation} 
y= w_0 + \mathbf w^T \phi\mathbf(x) 
\end{equation} 

            where, 
$ \phi(\mathbf x)$ is a polynomial feature transformation.

* We can tackle issues of overfitting by using ridge or lasso regularizaiton just like linear regression model.

#### **Label encoding**

Since the output $y$ is a discrete quantity, we use *one-hot encoding* to represent label. 

For a **binary classification :** 
* The label $0$ is represented with $[1,0]$, and 

* The label $1$ is represented with $[0,1]$.

The same scheme is extended to the **multi-class setting**. In general for a $k$ class set up, we use one hot encoding in $k$ components vector. $[y_1,y_2,\ldots, y_k]$ for label $1 \le r \le k, y_r$ would be 1 and other components would be 0.

Concretely for a **three class classification set up :**
* The label $0$ is represented with $[1,0,0]$

* The label $1$ is represented with $[0,1,0]$

* The label $2$ is represented with $[0,0,1]$

In the following class, we implement LabelTransformer, that converts discrete labels into `one-hot-encoding`.


In [1]:
import numpy as np 

In [2]:
class LabelTransformer(object):
    """ Label encoder decoder 
    Atrributes
    ----------
    n_classes : int 
        number of classes, K
    
    """

    def __init__(self, n_classes: int = None):
        self.n_classes = n_classes

    @property
    def n_classes(self):
        return self.__n_classes

    @n_classes.setter
    def n_classes(self, K):
        self.__n_classes = K
        self.__encoder = None if K is None else np.eye(K)

    @property
    def encoder(self):
        return self.__encoder

    def encode(self, class_indices: np.ndarray):
        """
        encode class index into one-of-k code 
        Parameters
        ----------
        class_indices : (N,) np.ndarray 
            non-negative class index 
            elements must be integer in [0, n_classes]


        Returns : 
        -------
        (N,K) np.ndarray 
            one-of-K encoding of input 

        """
        if self.n_classes is None:
            self.n_classes = np.max(class_indices) + 1

        return self.encoder[class_indices]

    def decode(self, onehot: np.ndarray):
        """
        decode one-of-k code into class index 
        parameters
        ----------
        onehot :(N,K) np.ndarray   
        Returns:
        -------
        (N,) np.ndarray 
            class index 
        """
        return np.argmax(onehot, axis=1)


#### **Demonstration of LabelTransformer**
##### 1. **Binary Classification setup** :

In [3]:
binary_labels = LabelTransformer(2).encode(np.array([1, 0, 0, 1]))
binary_labels

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [4]:
LabelTransformer().decode(binary_labels)

array([1, 0, 0, 1], dtype=int64)

##### 2. **Multiclassification set up with three classes**: 

In [5]:
multiclass_labels = LabelTransformer(4).encode(np.array([1, 2, 3, 1]))
multiclass_labels

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [6]:
LabelTransformer().decode(multiclass_labels)

array([1, 2, 3, 1], dtype=int64)